In [64]:
import pandas as pd
import numpy as np
import sys, os, re
import plotly.express as px

In [65]:
code_country_df = pd.read_csv('vessels_by_nationalities/code_country.tsv', sep="\t")
code_country_df = code_country_df.set_index('code')
code_country_temp = code_country_df.to_dict('index')
code_country = {c:code_country_temp[c]['country'] for c in code_country_temp}


In [77]:
#months = {"Jan": 1, "Feb": 2, "Mar": 3, "Apr": 4, "Apl": 4, "May": 5, "Jun": 6, "Jul": 7, "Aug": 8, "Sep": 9, "Oct": 10, "Nov": 11, "Dec": 12}

rx_time = re.compile(r'(.+?)-([A-Z][a-z]+)$')

#test_list = [['a','b','c'], ['AA','BB','CC']]
#pd.DataFrame(test_list, columns=['col_A', 'col_B', 'col_C'])

def read_vessel_data(file_path):
    data = pd.read_csv(file_path, skiprows=1)
    #df.replace({"col1": di})
    data["Country"].replace(code_country, inplace=True)

    #new_df = pd.DataFrame(columns = ["time", "country", "variable", "value"])
    temp_list = []
    for _, row in data.iterrows():
        for c in data.columns:
            if c not in ["Year", "Country"]:
                search_time = rx_time.search(c)

                if search_time:
                    time = search_time.group(2)
                    item = search_time.group(1).strip()
                    
                    if time == "Apl":
                        time = "Apr"
                    
                    if time != "Year":
                        temp_list_item = [str(row["Year"]) + " " + time, row["Country"], item, row[c]]
                        temp_list.append(temp_list_item)
                        #new_df.loc[len(new_df)] = temp_dic 
    
    new_df = pd.DataFrame(temp_list, columns = ["time", "country", "variable", "value"])
    new_df["time"] = pd.to_datetime(new_df["time"], format='%Y %b')
    return new_df

In [78]:
read_vessel_data("vessels_by_nationalities/ik-500h2021001.csv")

,time,country,variable,value
0,2021-01-01,100,Number Of Vessels,473
1,2021-01-01,100,Number Of Vessel (Direct Entrance),223
2,2021-01-01,100,Net Tonnages,7721086
3,2021-01-01,100,Net Tonnages (Direct Entrance),4076700
4,2021-02-01,100,Number Of Vessels,426
...,...,...,...,...
475,2021-01-01,Palau,Net Tonnages (Direct Entrance),7645
476,2021-02-01,Palau,Number Of Vessels,12
477,2021-02-01,Palau,Number Of Vessel (Direct Entrance),6
478,2021-02-01,Palau,Net Tonnages,23506


In [84]:

top_folder = "/Users/dgg32/Downloads/Japan_stats/vessels_by_nationalities/"
extension = ".csv"

df = pd.DataFrame(columns = ["time", "country", "variable", "value"])

for (head, dirs, files) in os.walk(top_folder):
    for file in files:
        if file.endswith(extension):
            current_file_path = os.path.abspath(os.path.dirname(os.path.join(head, file)))
            with_name = current_file_path + "/"+ file
            temp = read_vessel_data(with_name)
            df = pd.concat([df, temp])



In [85]:
df[df["country"] == "People's Republic of China"]["time"].dt.month.value_counts()

2     52
1     52
12    48
11    48
10    48
9     48
8     48
7     48
6     48
5     48
4     48
3     48
Name: time, dtype: int64

In [86]:
df.to_csv("vessels_melt.tsv", sep="\t", index=False)

In [92]:
def plot_time_trend(temp_df, title):
    fig = px.line(temp_df, x="time", y="value", title=title)
    fig.show()
    
    

In [93]:
plot_time_trend(df[(df["country"] == "People's Republic of China") & (df["variable"] == "Net Tonnages")].sort_values(["time"]), "Net Tonnages from China in Japan over time")

In [95]:
plot_time_trend(df[(df["country"] == 100) & (df["variable"] == "Net Tonnages")].sort_values(["time"]), "Net Tonnages from China in Japan over time")

In [96]:
plot_time_trend(df[(df["country"] == "Republic of Korea") & (df["variable"] == "Net Tonnages")].sort_values(["time"]), "Net Tonnages from China in Japan over time")

In [97]:
df.head()

,time,country,variable,value
0,2010-01-01,100,Number Of Vessels,266
1,2010-01-01,100,Number Of Vessel (Direct Entrance),185
2,2010-01-01,100,Net Tonnages,4057409
3,2010-01-01,100,Net Tonnages (Direct Entrance),2695172
4,2010-02-01,100,Number Of Vessels,239


In [116]:
df[(df["country"] != 100) & (df["variable"] == "Net Tonnages")][["time", "value"]].groupby(["time"]).sum("value").reset_index()

,time,value
0,2009-01-01,72167652
1,2009-02-01,66952356
2,2009-03-01,73139969
3,2009-04-01,67836524
4,2009-05-01,68065834
...,...,...
141,2020-10-01,68898852
142,2020-11-01,68882920
143,2020-12-01,72875169
144,2021-01-01,70981638


In [117]:
plot_time_trend(df[(df["country"] != 100) & (df["variable"] == "Net Tonnages")][["time", "value"]].groupby(["time"]).sum("value").reset_index(), "sum")